In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
from google.colab import drive
from google.colab import files

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import VGG16

# from GitHub

In [ ]:
drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive'
os.chdir(folder_path)



In [ ]:
import zipfile

with zipfile.ZipFile('PeopleArt-master.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

os.listdir('/content')

In [ ]:
styles = os.listdir('/content/PeopleArt-master/JPEGImages')

In [ ]:
styles

In [ ]:
print(f'terdapat {len(styles)} style art')

In [ ]:
directory_path = '/content/PeopleArt-master/JPEGImages'

jumlah_file_per_style = [(style, len(os.listdir(os.path.join(directory_path, style)))) for style in styles]

# Mengurutkan tuple berdasarkan jumlah_file dari yang terbanyak ke yang tersedikit
sorted_styles = sorted(jumlah_file_per_style, key=lambda x: x[1], reverse=True)

# Menampilkan hasil
for style, jumlah_file in sorted_styles:
    print(f"Jumlah file dalam folder {style}: {jumlah_file}")

In [ ]:
# Menghapus 10 folder dengan jumlah file paling sedikit
for folder, jumlah_file in sorted_styles[-16:]:
    folder_path = os.path.join(directory_path, folder)
    print(f"Menghapus folder {folder} dengan isi file {jumlah_file}")
    shutil.rmtree(folder_path)

In [ ]:
OUTPUT = len(os.listdir('/content/PeopleArt-master/JPEGImages'))
print(OUTPUT)
TRAINING_DIR = "/content/PeopleArt-master/JPEGImages"
training_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=40,
        shear_range=0.2,
        width_shift_range=0.3,
        height_shift_range=0.3,
        validation_split=0.3
)

validation_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.3
)

train_generator = training_datagen.flow_from_directory(
        directory=TRAINING_DIR,
        target_size=(224, 224),
        subset='training',
        class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
        directory=TRAINING_DIR,
        target_size=(224, 224),
        subset='validation',
        class_mode='categorical'
)

In [ ]:
# pakai vggnet16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.regularizers import l1, l2
model = tf.keras.models.Sequential([
    base_model,
    # tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(64, 64, 3)),
    # tf.keras.layers.MaxPooling2D(2, 2),
    # tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    # tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(OUTPUT, activation='softmax'),
])

model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
)

model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Tentukan callback ModelCheckpoint
checkpoint_path = '/content/best_model_epoch_{epoch:02d}_acc_{accuracy:.2f}_valaccuracy_{val_accuracy:.2f}.h5'
model_checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

# Kemudian, saat melatih model, sertakan callback ini dalam fungsi fit
history = model.fit(
    train_generator,
    epochs=150,
    validation_data=validation_generator,
    callbacks=[model_checkpoint]

    # callbacks=[callback]
    # steps_per_epoch=len(train_generator),
    # validation_steps=len(validation_generator)
  )